In [1]:
import zipfile

zip_file_path = '/content/drive/MyDrive/Consumer Complaint Classification/complaints.zip'
extract_path = '/content/drive/MyDrive/Consumer Complaint Classification/'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
import nltk
import re
from nltk.corpus import stopwords
import string

In [2]:
data = pd.read_csv("/content/drive/MyDrive/Consumer Complaint Classification/complaints/consumercomplaints.csv")
print(data.head())

   Unnamed: 0 Date received  \
0           0    2022-11-11   
1           1    2022-11-23   
2           2    2022-11-16   
3           3    2022-11-15   
4           4    2022-11-07   

                                             Product  \
0                                           Mortgage   
1  Credit reporting, credit repair services, or o...   
2                                           Mortgage   
3                        Checking or savings account   
4                                           Mortgage   

                  Sub-product                           Issue  \
0  Conventional home mortgage  Trouble during payment process   
1            Credit reporting     Improper use of your report   
2                 VA mortgage  Trouble during payment process   
3            Checking account             Managing an account   
4      Other type of mortgage  Trouble during payment process   

                                       Sub-issue  \
0                                

In [3]:
data = data.drop("Unnamed: 0",axis=1)

In [4]:
data.isnull().sum()

Date received                         0
Product                               0
Sub-product                      235294
Issue                                 0
Sub-issue                        683355
Consumer complaint narrative    1987977
dtype: int64

In [5]:
data.dropna()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative
3,2022-11-15,Checking or savings account,Checking account,Managing an account,Fee problem,"Hi, I have been banking with Wells Fargo for o..."
11,2022-11-09,Debt collection,Other debt,False statements or representation,Indicated you were committing crime by not pay...,XXXX is attempting to collect funds for Valuat...
15,2022-11-14,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account information incorrect,Today I called to get my balance and reset my ...
51,2022-10-12,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,The Federal Trade Commission Bureau of Consume...
72,2022-10-09,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Difficulty submitting a dispute or getting inf...,Ive mailed police report called been hung up o...
...,...,...,...,...,...,...
3101950,2017-03-04,Debt collection,I do not know,Disclosure verification of debt,Not given enough info to verify debt,I have received calls and notices in regards t...
3101955,2017-01-19,Student loan,Non-federal student loan,Can't repay my loan,Can't decrease my monthly payments,"Insanely high monthly payments, with "" no opti..."
3101956,2017-01-22,Student loan,Federal student loan servicing,Dealing with my lender or servicer,Need information about my balance/terms,My loans have an extraordinarily high interest...
3101958,2017-01-26,Debt collection,Auto,Communication tactics,Called after sent written cease of comm,Received cease and desist letter from them sho...


In [6]:
print(data["Product"].value_counts())

Credit reporting, credit repair services, or other personal consumer reports    1432096
Debt collection                                                                  452620
Mortgage                                                                         365181
Credit card or prepaid card                                                      165452
Checking or savings account                                                      141849
Credit reporting                                                                 140430
Credit card                                                                       89190
Bank account or service                                                           86206
Student loan                                                                      71530
Money transfer, virtual currency, or money service                                47641
Vehicle loan or lease                                                             37161
Consumer Loan                   

In [7]:
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
stopword=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
data["Consumer complaint narrative"] = data["Consumer complaint narrative"].apply(clean)

In [13]:
data = data[["Consumer complaint narrative", "Product"]]
x = np.array(data["Consumer complaint narrative"])
y = np.array(data["Product"])

cv = CountVectorizer()
X = cv.fit_transform(x)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.33,
                                                    random_state=42)

In [14]:
sgdmodel = SGDClassifier()
sgdmodel.fit(X_train,y_train)

SGDClassifier()

In [ ]:
user = input("Enter a Text: ")
data = cv.transform([user]).toarray()
output = sgdmodel.predict(data)
print(output)